In [87]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import ipywidgets as widgets
from ipywidgets import Output, HBox, VBox

from IPython.display import display, clear_output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from uiocat.analysis import GC_Analysis
from uiocat.reactions import Reaction
from uiocat.instrument import CoFeedRig, HighPressureRig
import datetime as dt
import os
import glob

plt.style.use('bmh')
plt.rcParams["figure.figsize"] = (6, 4)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# UiO Catalysis Catalytic Testing

## Data Analysis Dashbord

This dashbord analysis your "time-on-stream" data and calculates conversion, yield, and selectivity.
You can export the results to an excel report by clicking "Export to Excel" <br>
The saved excel report will be located in the same directory as your data.

In [88]:
#### COFEED RIG LAYOUT


layout = {'width': '700px'}  # Setting the widgets layout
style = {'description_width': '100px'}  # Setting the widget style

# The widget for raw data path
cofeed_input_folder = widgets.Text(
    placeholder='Folder Path',
    description='Directory',
    disabled=False,
    layout=layout, style=style)

# The widget for MS file name
cofeed_file = widgets.Select(
    description='Data files',
    disabled=False,
    layout=layout, style=style)

cofeed_button_analysis = widgets.Button(description="Run Analysis")
cofeed_output_analysis = widgets.Output()

cofeed_button_report = widgets.Button(description="Export to Excel")
cofeed_output_report = widgets.Output()

cofeed_button_clear = widgets.Button(description="Clear Window")

cofeed_Buttons = widgets.HBox([cofeed_button_analysis, cofeed_button_report, cofeed_button_clear]
                             , layout=widgets.Layout(display='flex', flex_flow='row', padding='10px 0 0 175px'))
cofeed_Outputs = widgets.VBox([cofeed_output_report, cofeed_output_analysis])

cofeed_ButtonSegment = VBox([cofeed_Buttons, cofeed_Outputs])

def on_cofeed_dir_change(change):
    files = os.listdir(change.new)
    files_csv = [f for f in files if '.csv' in f]
    cofeed_file.options = files_csv
    
    return

def cofeed_on_button_clicked(b):
    global cofeed_analysis
    
    with cofeed_output_analysis:
        clear_output()
        if (cofeed_file.value == '') or (cofeed_input_folder.value == ''):
            print("Insert file and folder paths!!")
        else:
            File_GC = os.path.join(cofeed_input_folder.value, cofeed_file.value)
            MTH = Reaction(reac='mth')  # First we define the reaction
            Co_Feed = CoFeedRig(File_GC, measurment_offset=5)  # Load instrument which contains instrument specific data such as respons factors
            cofeed_analysis = GC_Analysis(MTH, Co_Feed)  # Then we load the reaction, instrument, and data file
            cofeed_analysis.results()
            
def cofeed_on_report_button_clicked(b):
    with cofeed_output_report:
        clear_output()
        cofeed_analysis.export_to_excel()

def clear_output_button(b):
    with cofeed_output_analysis:
        clear_output()


cofeed_input_folder.observe(on_cofeed_dir_change, names='value')

cofeed_button_analysis.on_click(cofeed_on_button_clicked)
cofeed_button_report.on_click(cofeed_on_report_button_clicked)
cofeed_button_clear.on_click(clear_output_button)

# Setting up the display layout of the widgets
cofeed_Data_Paths_Widget = VBox([cofeed_input_folder, cofeed_file, cofeed_ButtonSegment])
#display(cofeed_Data_Paths_Widget)


In [89]:
#### HIGH PRESSURE RIG LAYOUT

layout = {'width': '700px'}  # Setting the widgets layout
style = {'description_width': '100px'}  # Setting the widget style

# The widget for raw data path
HighP_main_input_folder = widgets.Text(
    placeholder='Folder Path',
    description='Directory',
    disabled=False,
    layout=layout, style=style)

GC_file_Mid = widgets.Select(
    description='Mid FID',
    disabled=False,
    layout=layout, style=style)

GC_file_Rear = widgets.Select(
    description='Rear FID',
    disabled=False,
    layout=layout, style=style)


HighP_button_analysis = widgets.Button(description="Run Analysis")
HighP_output_analysis = widgets.Output()

HighP_button_report = widgets.Button(description="Export to Excel")
HighP_output_report = widgets.Output()

HighP_button_clear = widgets.Button(description="Clear Window")


HighP_Buttons = widgets.HBox([HighP_button_analysis, HighP_button_report, HighP_button_clear],
                             layout=widgets.Layout(display='flex', flex_flow='row', padding='10px 0 0 175px'))
HighP_Outputs = widgets.VBox([HighP_output_report, HighP_output_analysis])

HighP_ButtonSegment = VBox([HighP_Buttons, HighP_Outputs])

def on_HighP_dir_change(change):
    files = os.listdir(change.new)
    files_xls = [f for f in files if '.xls' in f]
    GC_file_Mid.options = files_xls
    GC_file_Rear.options = files_xls
    
    return
    

def HighP_on_button_clicked(b):
    global HighP_analysis
    
    with HighP_output_analysis:
        clear_output()
        if (GC_file_Mid.value == '') or (GC_file_Rear.value == '') or (HighP_main_input_folder.value == ''):
            print("Insert file and folder paths!!")
        else:
            FID_Files = [os.path.join(HighP_main_input_folder.value, GC_file_Mid.value), os.path.join(HighP_main_input_folder.value, GC_file_Rear.value)]
            MTH = Reaction(reac='mth-highP')  # First we define the reaction
            HighP_Rig = HighPressureRig(FID_Files)  # Load instrument which contains instrument specific data such as respons factors
            HighP_analysis = GC_Analysis(MTH, HighP_Rig)  # Then we load the reaction, instrument, and data file
            HighP_analysis.results()
            
def HighP_on_report_button_clicked(b):
    with HighP_output_report:
        clear_output()
        HighP_analysis.export_to_excel()
        
def HighP_clear_output_button(b):
    with HighP_output_analysis:
        clear_output()

        
HighP_main_input_folder.observe(on_HighP_dir_change, names='value')
HighP_button_analysis.on_click(HighP_on_button_clicked)
HighP_button_report.on_click(HighP_on_report_button_clicked)
HighP_button_clear.on_click(HighP_clear_output_button)

# Setting up the display layout of the widgets
HihgP_Data_Paths_Widget = VBox([HighP_main_input_folder, GC_file_Mid, GC_file_Rear, HighP_ButtonSegment])
#display(HihgP_Data_Paths_Widget)

In [90]:
tab_contents = ['CoFeed Rig', 'High Pressure Rig']
children = [cofeed_Data_Paths_Widget, HihgP_Data_Paths_Widget]
tab = widgets.Tab()
tab.children = children
[tab.set_title(i, title) for i, title in enumerate(tab_contents)]
tab